In [ ]:
import torch
import torch.nn as nn

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import fastf1 as ff1

# Habilitar  caché en una carpeta del proyecto
ff1.Cache.enable_cache("./cache")

1) Division de los Datos

Vamos a definir a continuacion que features del dataset podrian ser utiles para entrenar nuestro modelo. Con este analisis pretendemos hacer una primera division entre features probablemente utiles y features peligrosas que en principio descartariamos para no generar dataleakage

Empecemos por cargar la carrera de Monza 2025 (Italian Grand Prix)

In [19]:
year = 2025
gp_name = "Italian Grand Prix"   # Monza
session_name = "R"               # Race

session = ff1.get_session(year, gp_name, session_name)
session.load()

laps = session.laps


core           INFO 	Loading data for Italian Grand Prix - Race [v3.6.1]
req            INFO 	No cached data found for session_info. Loading data...
_api           INFO 	Fetching session info data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for driver_info. Loading data...
_api           INFO 	Fetching driver list...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for session_status_data. Loading data...
_api           INFO 	Fetching session status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for lap_count. Loading data...
_api           INFO 	Fetching lap count data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for track_status_data. Loading data...
_api           INFO 	Fetching track status data...
req            INFO 	Data has been written to cache!
req            INFO 	No ca

Veamos como esta compuesto el dataset:

In [20]:
print("Datos disponibles por cada vuelta:\n")
# Mostrar prolijamente las columnas
for col in laps.columns: 
    print(col)
    

Datos disponibles por cada vuelta:

Time
Driver
DriverNumber
LapTime
LapNumber
Stint
PitOutTime
PitInTime
Sector1Time
Sector2Time
Sector3Time
Sector1SessionTime
Sector2SessionTime
Sector3SessionTime
SpeedI1
SpeedI2
SpeedFL
SpeedST
IsPersonalBest
Compound
TyreLife
FreshTyre
Team
LapStartTime
LapStartDate
TrackStatus
Position
Deleted
DeletedReason
FastF1Generated
IsAccurate


Viendo la lista de datos:
- Laptime es nuestro Target

Podemos descartar rapidamente para evitar dataleakeage:
- Sector1Time, Sector2Time y Sector3Time ya que no son mas que el Target descompuesto en 3 tiempos.
- IsPersonalBest tambien la podemos descartar ya que usa info de todas las vueltas del piloto y depende de saber cuales fueron las mas rapidas

Por ser tiempos absolutos de sesión (los podriamos usar más adelante como features derivados, pero no crudos):
- Time
- Sector1SessionTime, Sector2SessionTime, Sector3SessionTime
- LapStartTime, LapStartDate
- PitInTime, PitOutTime

Por depender de telemetría dentro de la vuelta y qyue no es conocida antes de correrla:
- SpeedI1, SpeedI2, SpeedFL, SpeedST (speed traps durante la vuelta).

Por ser meta-datos o cosas mas raras:
- Deleted, DeletedReason, IsAccurate (sirven para filtrar vueltas malas, no para predecir)
- TyresNotChanged, LapFlags, LapCountTime, StartLaps, Outlap, etc. (campos poco claros / sin documentación fuerte, los ignoramos para empezar)


In [21]:
# Me las guardo para despues
FEATURES_DROP_SIM = [
    "LapTime",
    "Sector1Time", "Sector2Time", "Sector3Time",
    "Sector1SessionTime", "Sector2SessionTime", "Sector3SessionTime",
    "Time", "LapStartTime", "LapStartDate",
    "PitInTime", "PitOutTime",
    "SpeedI1", "SpeedI2", "SpeedFL", "SpeedST",
    "IsPersonalBest",
    "Deleted", "DeletedReason", "IsAccurate",
    "TyresNotChanged", "LapFlags", "LapCountTime", "StartLaps", "Outlap",
]


Filtro solo por las vueltas de Colapinto

In [22]:
# 2) Filtrar las vueltas de Franco Colapinto
driver_code = "COL"

col_laps = laps[
    (laps["Driver"] == driver_code)
    & laps["LapTime"].notna()
    & (~laps.get("Deleted", False))   # si 'Deleted' existe y es True, la sacamos
]

# Ordenamos por número de vuelta por prolijidad
col_laps = col_laps.sort_values("LapNumber").reset_index(drop=True)

# 3) Vector Y: tiempos de vuelta (en segundos)
Y = col_laps["LapTime"].dt.total_seconds().to_numpy()

print(f"Nº de vueltas de Colapinto en carrera: {len(Y)}")
print("Primeros 5 tiempos (s):", Y[:5])


Nº de vueltas de Colapinto en carrera: 49
Primeros 5 tiempos (s): [94.322 86.775 85.346 85.04  84.945]


Ahora como primera aproximacion basica voy a usar los siguientes features para mi X:

Datos de vuelta:
- LapNumber: Índice de vuelta en carrera → captura efecto de peso de combustible que baja, evolución de pista, etc.
- Stint: Número de stint → correlaciona con cambio de compuesto / momento de la carrera.

Datos de Neumáticos:
- Compound (categórica: SOFT / MEDIUM / HARD / INTERMEDIATE / WET)
- TyreLife (float: vueltas que tiene ese neumático)
- FreshTyre (bool: si el set era nuevo al inicio del stint)


Datos de Contexto de carrera:
- TrackStatus (string con códigos de estado de pista: verde, SC, VSC, etc.)
- Position (posición en carrera al final de la vuelta; para el simulador la vamos a aproximar como posición al inicio de la siguiente vuelta).


In [23]:
FEATURES_BASIC = [
    "LapNumber",
    "Stint",
    "Compound",
    "TyreLife",
    "FreshTyre",
    "TrackStatus",
    "Position",
]

# Nos quedamos solo con las que efectivamente existen en el DF por si faltara alguna
FEATURES_BASIC = [c for c in FEATURES_BASIC if c in col_laps.columns]

X = col_laps[FEATURES_BASIC].copy()
X.head()


,LapNumber,Stint,Compound,TyreLife,FreshTyre,TrackStatus,Position
0,1.0,1.0,MEDIUM,1.0,True,1,17.0
1,2.0,1.0,MEDIUM,2.0,True,1,17.0
2,3.0,1.0,MEDIUM,3.0,True,1,17.0
3,4.0,1.0,MEDIUM,4.0,True,1,17.0
4,5.0,1.0,MEDIUM,5.0,True,1,17.0


2) Primer Analisis Exploratorio de Datos

In [ ]:
# Columna de LapTime en segundos
col_laps = col_laps.copy()
col_laps["LapTime_s"] = col_laps["LapTime"].dt.total_seconds()

# Por prolijidad, nos quedamos con un subconjunto de columnas relevantes para EDA
cols_eda = [
    "LapNumber", "LapTime", "LapTime_s",
    "Stint", "Compound", "TyreLife",
    "TrackStatus", "Position"
]
cols_eda = [c for c in cols_eda if c in col_laps.columns]

col_laps[cols_eda].head()
col_laps[["LapTime_s", "TyreLife"]].describe()


Veamos una evolución del tiempo de vuelta a lo largo de la carrera

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 4))
plt.plot(col_laps["LapNumber"], col_laps["LapTime_s"], marker="o")
plt.xlabel("Número de vuelta")
plt.ylabel("Tiempo de vuelta [s]")
plt.title("Evolución del tiempo de vuelta - Colapinto (Monza 2025)")
plt.grid(True)
plt.show()


Histograma de tiempos de vuelta

In [ ]:
plt.figure(figsize=(6, 4))
plt.hist(col_laps["LapTime_s"], bins=15, edgecolor="black")
plt.xlabel("Tiempo de vuelta [s]")
plt.ylabel("Frecuencia")
plt.title("Distribución de tiempos de vuelta - Colapinto")
plt.grid(axis="y")
plt.show()


Tiempo de vuelta vs número de stint (cambio de neumáticos)

In [ ]:
if "Stint" in col_laps.columns:
    plt.figure(figsize=(8, 4))
    for stint, df_stint in col_laps.groupby("Stint"):
        plt.plot(df_stint["LapNumber"], df_stint["LapTime_s"],
                 marker="o", linestyle="-", label=f"Stint {stint}")
    plt.xlabel("Número de vuelta")
    plt.ylabel("Tiempo de vuelta [s]")
    plt.title("LapTime por stint - Colapinto")
    plt.legend()
    plt.grid(True)
    plt.show()


Relación entre TyreLife y tiempo de vuelta

In [ ]:
if "TyreLife" in col_laps.columns and "Compound" in col_laps.columns:
    plt.figure(figsize=(8, 5))

    compounds = col_laps["Compound"].dropna().unique()
    for comp in compounds:
        df_c = col_laps[col_laps["Compound"] == comp]
        plt.scatter(df_c["TyreLife"], df_c["LapTime_s"], alpha=0.7, label=comp)

    plt.xlabel("TyreLife [vueltas]")
    plt.ylabel("LapTime [s]")
    plt.title("LapTime vs TyreLife por compuesto - Colapinto")
    plt.legend(title="Compound")
    plt.grid(True)
    plt.show()

Posición en carrera vs vuelta (race trace)

In [ ]:
if "Position" in col_laps.columns:
    plt.figure(figsize=(8, 4))
    plt.plot(col_laps["LapNumber"], col_laps["Position"], marker="o")
    plt.xlabel("Número de vuelta")
    plt.ylabel("Posición en carrera")
    plt.title("Evolución de la posición - Colapinto")
    plt.gca().invert_yaxis()  # posición 1 arriba, 20 abajo
    plt.grid(True)
    plt.show()


TrackStatus vs LapNumber

In [ ]:
if "TrackStatus" in col_laps.columns:
    # Mapeamos cada TrackStatus a un número
    status_codes = {st: i for i, st in enumerate(col_laps["TrackStatus"].astype(str).unique())}
    col_laps["TrackStatus_code"] = col_laps["TrackStatus"].astype(str).map(status_codes)

    plt.figure(figsize=(10, 4))
    plt.scatter(col_laps["LapNumber"], col_laps["LapTime_s"],
                c=col_laps["TrackStatus_code"], cmap="tab10")
    cbar = plt.colorbar()
    cbar.set_ticks(list(status_codes.values()))
    cbar.set_ticklabels(list(status_codes.keys()))
    plt.xlabel("Número de vuelta")
    plt.ylabel("LapTime [s]")
    plt.title("LapTime coloreado por TrackStatus")
    plt.grid(True)
    plt.show()


PROXIMOS PASOS:

- Detectar outliers de LapTime (vueltas raras que quizás conviene sacar).

- Entender cómo influye el neumático y su vida en el tiempo de vuelta.

- Ver si el ritmo cambia mucho entre stints.

- Ver la historia de la carrera de Colapinto (posición, laps bajo SC, etc.).

PARA PENSAR:

- Que otras formas de armar el set de train podemos usar? (Carrera+FP1+FP2+FP3 / Solo Carrera / Usar solo Carrera pero usar todos los Circuitos del calendario / Usar todo de todo)

- Que modelos vamos a usar?